# Advance Through an Array

In a particular board game, a player has to advance through a sequence of positions. Each positions has a nonnegative integer associated with it, representing the maximum you can advance from that osition in one move. You begin at the first position, and win by getting to the last position.

For example, let A = [3, 3, 1, 0, 2, 0, 1] represent the board game, i.e., the ith entry in A is the maximum we an advance from i, Then the game can be won by the following sequence of teh advances through A: A[0] (3) → A[1] (3) → A[4] (2) → A[6]

If A was [3, 2, 0, 0, 2, 0, 1], it would not possible to advance past position A[0] (3) so the game cannot be won. This is because A[0] (3) → A[1] (2) → 0, A[0] (3) → A[2] (0) → 0 and A[0] (3) → A[3] (0) → 0.

## Abstract

Write a program which takes an array of integers, which A[i] denotes the maximum you can advance from index i, and returns whether it is possible to advance to the last index starting from the beginning of the array.

Example 1:
```
Input: [3, 3, 1, 0, 2, 0, 1]
Ouput: True
Explaination: 3 -> 3 -> 2 -> 1
```

Example 2:
```
Input: [3, 2, 0, 0, 2, 0, 1]
Output: False
Explaination: Cannot further advance after 3. 
```

Example 3:
```
Input: [0]
Output: True
Explaination: The start point is already the last index.
```

**Hint**: Analyze each location, starting from the beginning.

## Solution: Furthest to go at each index

It is natrual to try advancing as far as possible in each step. This approach does not always work, because it potentially skips indices containing large entries. For example, if A = [2, 4, 1, 1, 0, 2, 3], advancing from A[0] furthest 2 steps goes to A[2] (1) → A[3] (1) → A[4] (0), after which it cannot progress. However, advancing to index 1, which contains a 4 lets us proceed to index 5, from which we can advance to index 6.

The above example suggests iterating through ::all entries:: in A. As we iterate through the array, we track the furthest index we know we can advance to, which is the max of the furthest index to reach at index i-1, and i + A[i].

For example, if A = [3, 3, 1, 0, 2, 0, 1], we iteratively compute the furthest we can advance at each index i.

| **i**     | 0    | 1    | 2    | 3    | 4    | 5    | 6    |
| ---------- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| **A[i]**   | 3    | 3    | 1    | 0    | 2    | 0    | 1    |
| **i + A[i]** | 3    | 4    | 2    | 3    | 6    | 5    | 7    |
| **furthest** | 3    | 4    | 4    | 4    | 6    |      |      |

The `furthest` denotes the furthest position this index could reach, which has a default value of 0.

If A = [3, 2, 0, 0, 2, 0, 1], we iteratively update the furthest we can advance to as 0, 3, 3, 3, 3, after which we cannot advance, so it is not possible to reach the last index.

| **i**        | 0    | 1    | 2    | 3    | 4    | 5    | 6    |
| ------------ | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| **A[i]**     | 3    | 2    | 0    | 0    | 2    | 0    | 1    |
| **i + A[i]** | 3    | 3    | 2    | 3    | 6    | 5    | 7    |
| **furthest** | 3    | 3    | 3    | 3    |      |      |      |

Let f(i) denotes the furthest index at index i could advance:
* $f(i) = max(f(i-1), \ i+A[i])$ when $f(i-1) >= i$
* when $f(i-1) < i$, it is not possible to advance to position `[i]` so the program exit!
* f(-1) = 0 as default

**Time complexity**: $O(n)$  
**Space complexity**: $O(1)$

In [1]:
def can_reach_end(A: list) -> bool:
    reached_index, last_index = 0, len(A) - 1
    i = 0
    while reached_index < last_index and reached_index >= i:
        reached_index = max(reached_index, i + A[i])
        i += 1
    return reached_index >= last_index

In [2]:
assert can_reach_end([3, 3, 1, 0, 2, 0, 1]) is True
assert can_reach_end([3, 2, 0, 0, 2, 0, 1]) is False
assert can_reach_end([0]) is True

## Follow-ups: Compute the path of the minimum steps to reach the end.

Assuming it can reach the end index, display the indices with minimum steps to move to the end.

The algorithm is greedy, which is to choose the optimal index, which can advance furthest in the **next** move.

Taking [3, 3, 1, 0, 2, 0, 1] for example, index 0 is 3 (denote [0] (3)), which means the next steps it can advace are:
```
[1] (3) -> 4 *
[2] (1) -> 3
[3] (0) -> 3
```

Thus, index [1] with value 4 is the optimal of the next move. Then the next step from index [1] (4), select the next optimal index:
```
[2] (1) -> 3 -
[3] (0) -> 3 -
[4] (2) -> 6 *
[5] (0) -> 5
```

So the position with index [4] is the best, can it can reach index 6, which is the last index in this array. This approach lead the minimum stpes are index [0] -> [1] -> [4] -> [6].

We can optimize the algorithm by avoiding the duplicated compare (mark with `-` in the above steps). These two steps are caculated in the last step, so we just start from the next index of index [3], which is index [4].

In [3]:
def steps_to_reach_end(A: list) -> bool:
    steps = [0]
    while steps[-1] < len(A) - 1:
        last_reached_index = steps[-1]
        reaching_index_length = A[last_reached_index]
        if last_reached_index + reaching_index_length > len(A):
            steps.append(len(A) - 1)
            break
        B = [i + A[i] for i in range(last_reached_index, last_reached_index + reaching_index_length+1)]
        next_step = last_reached_index + B.index(max(B))
        steps.append(next_step)
    return steps

In [4]:
assert steps_to_reach_end([3, 3, 1, 0, 2, 0, 1]) == [0, 1, 4, 6]

Refs: [EPI 5.4](https://github.com/adnanaziz/EPIJudge/blob/master/epi_judge_python_solutions/advance_by_offsets.py)